In [1]:
import pyspark

from pyspark.sql import SparkSession

from pyspark.sql import types as T
import findspark 

findspark.init()

In [2]:
spark_session = SparkSession.builder.master('local[*]').appName('spark_assignment').getOrCreate()

23/03/05 17:58:41 WARN Utils: Your hostname, idowu-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
23/03/05 17:58:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 17:58:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#spark version
pyspark.__version__

'3.3.2'

In [ ]:
url = "https://opendata.reseaux-energies.fr/explore/dataset/eco2mix-national-tr/download/?format=csv"
from pyspark import SparkFiles
sc.addFile(url)
     
path  = SparkFiles.get('download')
df = spark.read.csv("file://" + path, header=True, inferSchema= True, sep = ";")

In [3]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz"
from pyspark import SparkFiles

spark_session.sparkContext.addFile(url)

# path = SparkFiles.get('download')

df_fhv = spark_session.read.csv("file://" + SparkFiles.get("fhvhv_tripdata_2021-06.csv.gz"), header=True, inferSchema=True, sep =",")

In [20]:
df_fhv.repartition(12).write.mode('overwrite').parquet('data/raw/fhv/2021/06')

23/03/05 17:23:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [7]:
# 24MB
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [5]:
from datetime import datetime

In [8]:
df_fhv.createOrReplaceTempView('fhv_june_data')

In [13]:
spark_session.sql("""
SELECT count(*)
FROM fhv_june_data
WHERE CAST(pickup_datetime AS DATE) BETWEEN '2021-06-15' AND '2021-06-15'
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [27]:
# question 4
spark_session.sql("""
SELECT MAX(datediff(hour,pickup_datetime, dropoff_datetime)),CAST(pickup_datetime AS DATE)
FROM fhv_june_data
GROUP BY CAST(pickup_datetime AS DATE)
ORDER BY 1 DESC
""").show()

+-----------------------------------------------------------+---------------+
|max(timestampdiff(hour, pickup_datetime, dropoff_datetime))|pickup_datetime|
+-----------------------------------------------------------+---------------+
|                                                         66|     2021-06-25|
|                                                         25|     2021-06-22|
|                                                         19|     2021-06-27|
|                                                         18|     2021-06-26|
|                                                         16|     2021-06-23|
|                                                         13|     2021-06-24|
|                                                         11|     2021-06-04|
|                                                         10|     2021-06-01|
|                                                         10|     2021-06-20|
|                                                          9|   

In [30]:
URL_ZONE = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

spark_session.sparkContext.addFile(URL_ZONE)

df_zone = spark_session.read.csv("file://"+SparkFiles.get('taxi_zone_lookup.csv'), header=True, inferSchema=True, sep=',')

23/03/05 18:36:02 WARN SparkContext: The path https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv has been added already. Overwriting of added paths is not supported in the current version.


In [31]:
df_zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [32]:
df_zone.createOrReplaceTempView('zone_lookup_table')



In [34]:
# question 6
spark_session.sql("""
WITH fhv_data AS (SELECT * 
FROM fhv_june_data),

zone_data AS (SELECT * FROM
zone_lookup_table)

SELECT pick_off_zone.zone, COUNT(*)
FROM 
fhv_data AS fd
INNER JOIN zone_lookup_table as pick_off_zone
ON fd.PULocationID = pick_off_zone.LocationID
GROUP BY 1
ORDER BY 2 DESC
""").show()


+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
| Crown Heights North|  231279|
|        East Village|  221244|
|         JFK Airport|  188867|
|      Bushwick South|  187929|
|       East New York|  186780|
|TriBeCa/Civic Center|  164344|
|   LaGuardia Airport|  161596|
|            Union Sq|  158937|
|        West Village|  154698|
|             Astoria|  152493|
|     Lower East Side|  151020|
|        East Chelsea|  147673|
|Central Harlem North|  146402|
|Williamsburg (Nor...|  143683|
|          Park Slope|  143594|
|  Stuyvesant Heights|  141427|
|        Clinton East|  139611|
|West Chelsea/Huds...|  139431|
|             Bedford|  138428|
|         Murray Hill|  137879|
+--------------------+--------+
only showing top 20 rows

